In [9]:
!pip install --quiet optuna

In [10]:
import numpy as np
import pandas as pd
import optuna as opt
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from pathlib import Path
import gc

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
RANDOM_SEED = 2
DATA_DIR = Path("/content/drive/MyDrive/CS760")

In [13]:
df_train = pd.read_parquet(DATA_DIR/"train_main.parquet.snappy")
df_test = pd.read_parquet(DATA_DIR/"test_main.parquet.snappy")

print(f"Shape of the training data : {df_train.shape}")
print(f"Shape of the test data : {df_test.shape}")

Shape of the training data : (400000, 13)
Shape of the test data : (100000, 13)


In [14]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
r_id,400000.0,3.525889e+06,2.018260e+06,1.900000e+01,1.807686e+06,3.486237e+06,5.294850e+06,6.990280e+06
r_stars,400000.0,3.607683e+00,1.539312e+00,1.000000e+00,2.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00
r_stars_square,400000.0,1.538485e+01,9.625009e+00,1.000000e+00,4.000000e+00,1.600000e+01,2.500000e+01,2.500000e+01
r_length,400000.0,1.332325e+02,1.149695e+02,0.000000e+00,5.700000e+01,1.010000e+02,1.700000e+02,1.068000e+03
u_friends_count,400000.0,1.899421e+02,5.735422e+02,1.000000e+00,2.000000e+00,4.000000e+01,1.680000e+02,1.499500e+04
u_review_count,400000.0,1.915879e+02,5.024456e+02,0.000000e+00,1.100000e+01,4.400000e+01,1.760000e+02,1.747300e+04
u_month_age,400000.0,4.277618e+01,3.456388e+01,3.802649e-07,1.362863e+01,3.651231e+01,6.514719e+01,2.017248e+02
b_stars,400000.0,3.744636e+00,7.930024e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
b_review_count,400000.0,3.346235e+02,6.589277e+02,5.000000e+00,3.900000e+01,1.250000e+02,3.410000e+02,7.568000e+03
r_sen,400000.0,1.821748e-01,1.927162e-01,-1.000000e+00,6.000000e-02,1.805560e-01,3.001218e-01,1.000000e+00


In [15]:
X_train, y_train = df_train.drop(['r_useful', 'r_id'], axis=1).values, df_train['r_useful'].values
X_test, y_test = df_test.drop(['r_useful', 'r_id'], axis=1).values, df_test['r_useful'].values

In [16]:
def objective(trial):
  """ Function to tune parameters """
  gc.collect()
  params = {
      "n_estimators":trial.suggest_int('n_estimators', 1, 1001, step=50),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "learning_rate" : trial.suggest_float('learning_rate', 0.001, 0.501, step=0.005),
      "reg_alpha": 10 ** trial.suggest_int("reg_alpha", -3, 3),
      "reg_lambda": 10 ** trial.suggest_int("reg_lambda", -3, 3),
      "subsample ": trial.suggest_float("subsample", 0.5, 1, step=0.1),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1, step=0.1)
  }

  print("Currently running with:")
  print(params)

  model = xgb.XGBRegressor(objective="reg:squarederror",
                            n_jobs=-1,
                            grow_policy='lossguide',
                            tree_method="gpu_hist",
                            predictor="gpu_predictor",
                            booster='gbtree',
                            sampling_method='gradient_based',
                            eval_metrics=['rmse'],
                            random_state=RANDOM_SEED,
                            enable_categorical=False,
                            **params)
  pipe = make_pipeline(StandardScaler(), model)

  scores = cross_val_score(pipe, X_train, y_train,
    scoring="neg_root_mean_squared_error", cv=KFold(3))

  return -scores.mean()

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params

[I 2022-09-27 10:22:18,011] A new study created in memory with name: no-name-3c19f5c0-f971-48f2-be52-7fea31f48685


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.276, 'reg_alpha': 1, 'reg_lambda': 0.1, 'subsample ': 0.6, 'colsample_bytree': 0.6}


[I 2022-09-27 10:22:22,210] Trial 0 finished with value: 3.649719144314228 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.276, 'reg_alpha': 0, 'reg_lambda': -1, 'subsample': 0.6, 'colsample_bytree': 0.6}. Best is trial 0 with value: 3.649719144314228.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.131, 'reg_alpha': 10, 'reg_lambda': 1, 'subsample ': 0.5, 'colsample_bytree': 0.8}


[I 2022-09-27 10:22:40,486] Trial 1 finished with value: 3.542791588341848 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.131, 'reg_alpha': 1, 'reg_lambda': 0, 'subsample': 0.5, 'colsample_bytree': 0.8}. Best is trial 1 with value: 3.542791588341848.


...

[I 2022-09-27 12:37:03,573] Trial 48 finished with value: 3.515844714611267 and parameters: {'n_estimators': 701, 'max_depth': 12, 'learning_rate': 0.021, 'reg_alpha': -1, 'reg_lambda': 1, 'subsample': 0.6, 'colsample_bytree': 0.9}. Best is trial 36 with value: 3.496549441410488.


Currently running with:
{'n_estimators': 601, 'max_depth': 14, 'learning_rate': 0.47600000000000003, 'reg_alpha': 10, 'reg_lambda': 10, 'subsample ': 0.7, 'colsample_bytree': 0.8}


[I 2022-09-27 12:39:51,944] Trial 49 finished with value: 3.717274842593086 and parameters: {'n_estimators': 601, 'max_depth': 14, 'learning_rate': 0.47600000000000003, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.7, 'colsample_bytree': 0.8}. Best is trial 36 with value: 3.496549441410488.


Final best parameters:


{'n_estimators': 651,
 'max_depth': 10,
 'learning_rate': 0.016,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 0.6,
 'colsample_bytree': 0.8}

In [17]:
best_params = study.best_params.copy()
best_params["reg_lambda"] = 10 ** best_params["reg_lambda"]
best_params["reg_alpha"] = 10 ** best_params["reg_alpha"]

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.8, enable_categorical=False,
                              eval_metrics=['rmse'], grow_policy='lossguide',
                              learning_rate=0.016, max_depth=10,
                              n_estimators=651, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              reg_alpha=1, reg_lambda=10,
                              sampling_method='gradient_based', subsample=0.6,
                              tree_method='gpu_hist'))])

In [18]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 2.8977865784650314, MAE: 1.3966921443385631
test results - RMSE: 4.066559773782579, MAE: 1.5604560980021953


In [19]:
mean_train = np.mean(y_train)
print(f"Baseline mean model - test RMSE: {np.sqrt(np.mean((y_test - mean_train)**2))}")
print(f"Baseline mean model - test MAE: {np.mean(np.abs(y_test - mean_train))}")

Baseline mean model - test RMSE: 4.685317144421096
Baseline mean model - test MAE: 1.9206922705000002
